In [40]:
import sys
import os
from datetime import datetime, timedelta
sys.path.append("../")

In [41]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import numpy as np
import plotly_template as pt
from functions import emissions, func, mp_funcs, speed
import definitions
import pandas as pd
import pickle
import math
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

# pio.renderers.default = "browser"

# Generating Iterations of the Heatmaps

### Creating the emissions plot function

### File locations and constants

In [42]:
DATA_FOLDER =['2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14']
DATA_DIR = r'Z:\6 ACTION Project\SUMO\Sim_Output_Data'
# DATA_DIR_FULL_PATH = os.path.join(definitions.DATA_DIR, DATA_FOLDER)
# DATA_DIR_FULL_PATH = os.path.join(DATA_DIR, DATA_FOLDER)
RAW_DATA_FILE = 'data.csv'
RAW_DATA_SUMMARY = 'data_summary.csv'
RAW_DATA_FILE_PARQUET = 'data.parquet'
RAW_DATA_SUMMARY_PARQUET = 'data_summary.parquet'

## Read in the Data

In [43]:
raw_data = {}
summary_data = {}
days_extended = 1
start_hour = "23:00:00.000"
for folder in DATA_FOLDER:
    DATA_DIR_FULL_PATH = os.path.join(DATA_DIR, folder)
    try:
        raw_data[folder] = pd.read_parquet(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_FILE_PARQUET),)
        summary_data[folder] = pd.read_parquet(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_SUMMARY_PARQUET),)

    except FileNotFoundError:
        
        date_low = (datetime.strptime(folder, "%Y-%m-%d") - timedelta(days=days_extended)).strftime("%m/%d/%Y")
        start_time = " ".join([date_low, start_hour])
        raw_data[folder] = pd.read_csv(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_FILE), low_memory=False, index_col=0)
        summary_data[folder] = pd.read_csv(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_SUMMARY), header=[0, 1], index_col=0)
        raw_data[folder].loc[:, 'timestep_time'] = mp_funcs.apply_by_multiprocessing(raw_data[folder]['timestep_time'], func, start_time=start_time)

        raw_data[folder].to_parquet(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_FILE_PARQUET))
        table = pa.Table.from_pandas(summary_data[folder])
        pq.write_table(table, os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_SUMMARY_PARQUET))
        # summary_data[folder].to_parquet(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_SUMMARY_PARQUET), index=True)

In [44]:
decel_emissions_dict = {}
for folder in DATA_FOLDER:
    raw_data[folder]['timestep_time'] = pd.to_datetime(raw_data[folder]['timestep_time'])
    decel_emissions_dict[folder] =  raw_data[folder].copy()
    mask = decel_emissions_dict[folder]['vehicle_fuel'] <= 0.000076
    decel_emissions_dict[folder].loc[mask, 'vehicle_fuel'] = 0.000077  # gallons per second

In [32]:
# mask = raw_data[folder]['vehicle_speed'] == 0
# raw_data[folder].loc[mask, 'vehicle_fuel']
decel_emissions_dict

{}

### Bining the Emissions Data

In [45]:
resample_period = '30T' # aka the bin size
resample_period_text = '30 min'
bin_size = 5

In [46]:
# df = pd.read_csv(os.path.join(DATA_DIR_FULL_PATH, RAW_DATA_FILE), low_memory=False, index_col=0)
binned_emissions_dict = {}
for folder in DATA_FOLDER:
    # raw_data[folder]['timestep_time'] = pd.to_datetime(raw_data[folder]['timestep_time'])
    emissions.bin_2D_sum(emissions_df=raw_data[folder], bin_column=['vehicle_fuel'], resample_period=resample_period, bin_size=5, processor_num=2)
    binned_emissions_dict[folder] = pickle.load(open(os.path.join(definitions.DATA_DIR, 'emissions_dict.pkl'), 'rb'))

func:bin_2D_sum took: 4.321222305297852 sec
func:bin_2D_sum took: 3.291393995285034 sec
func:bin_2D_sum took: 3.95157527923584 sec
func:bin_2D_sum took: 4.067893028259277 sec
func:bin_2D_sum took: 4.28631591796875 sec


In [47]:
for folder in DATA_FOLDER:
    # raw_data[folder]['timestep_time'] = pd.to_datetime(raw_data[folder]['timestep_time'])
    emissions.bin_2D_sum(emissions_df=decel_emissions_dict[folder], bin_column=['vehicle_fuel'], resample_period=resample_period, bin_size=5, processor_num=2)
    decel_emissions_dict[folder] = pickle.load(open(os.path.join(definitions.DATA_DIR, 'emissions_dict.pkl'), 'rb'))

func:bin_2D_sum took: 3.5176374912261963 sec
func:bin_2D_sum took: 3.3062586784362793 sec
func:bin_2D_sum took: 3.94419264793396 sec
func:bin_2D_sum took: 4.071604251861572 sec
func:bin_2D_sum took: 4.332248210906982 sec


## Plotting the Data

#### Plot Constants

In [48]:
plot_radius = 8
mapbox_key = "pk.eyJ1IjoibWF4LXNjaHJhZGVyIiwiYSI6ImNrOHQxZ2s3bDAwdXQzbG81NjZpZm96bDEifQ.etUi4OK4ozzaP_P8foZn_A"

colorscale = [
              [0.0, "rgba(0, 255, 204, 0)"],
              [0.2, "rgb(0, 255, 51)"],
              [0.4, "rgb(204, 255, 0)"],
              [0.6, "rgb(255, 204, 51)"],
              [0.8, "rgb(255, 102, 51)"],
              [1.0, "rgb(204,0,0)"],
            ]

color_axis = dict(
            cmin=0,
            cmax=None,
            showscale=True,
            colorscale=colorscale,
            colorbar=dict(
                outlinecolor="black",
                outlinewidth=2,
                ticks="outside",
                # tickfont=pt.font_dict,
                tickvals=None, # tickvalues,
                ticktext=None,
                title=f"[gal per {resample_period_text}]<br /> <br />",
                tickformat="e",
                # titlefont=pt.font_dict,
            )
)

In [49]:
plot_time_list = [['2020-02-10', pd.to_datetime('2020-02-10T07:30:00')], ['2020-02-10', pd.to_datetime('2020-02-10T07:30:00')]]
title_add_ons = [' No Decel Fuel Consumption', ' Decel Fuel Consumption Included']

num_plots = len(plot_time_list)
fig = make_subplots(rows=1, cols=num_plots,
                    specs=[[{'type': 'mapbox', }] * num_plots],
                    subplot_titles=[str(time[1]) + title_add_ons[i] for i, time in enumerate(plot_time_list)],
                    column_widths=[(1 - 0.1 * (num_plots / 2)) / num_plots]*num_plots,
                    horizontal_spacing=0.05,)

mapbox_dict = {'mapbox': dict(accesstoken=mapbox_key,
                                bearing=0,
                                style='mapbox://styles/max-schrader/ck8t1cmmc02wk1it9rv28iyte',
                                center=go.layout.mapbox.Center(
                                    lat=33.126275,
                                    lon=-87.54891
                                ),
                                pitch=0,
                                zoom=14.45),}

max_value = -20
for i, time in enumerate(plot_time_list):
    
    if i % 2 == 0:
    
        z_data = binned_emissions_dict[time[0]][time[1]]['data']
        lat_edges = binned_emissions_dict[time[0]][time[1]]['lat']
        lon_edges = binned_emissions_dict[time[0]][time[1]]['lon']
        max_value = binned_emissions_dict[time[0]][time[1]]['max_value'] if binned_emissions_dict[time[0]][time[1]]['max_value'] > max_value else max_value
    
    else:
        
        z_data = decel_emissions_dict[time[0]][time[1]]['data']
        lat_edges = decel_emissions_dict[time[0]][time[1]]['lat']
        lon_edges = decel_emissions_dict[time[0]][time[1]]['lon']
        max_value = decel_emissions_dict[time[0]][time[1]]['max_value'] if decel_emissions_dict[time[0]][time[1]]['max_value'] > max_value else max_value
        
    fig.add_trace(go.Densitymapbox(lat=lat_edges,
                                    lon=lon_edges,
                                    z=z_data,
                                    # customdata=summed_array[0][2],
                                    # hovertemplate='%{customdata}' if sum else None,
                                    radius=plot_radius,
                                    hoverinfo='z',
                                    coloraxis="coloraxis",
                                    ),
                    row=1,
                    col=i+1)
    
    mapbox_dict['mapbox'+ str(i+2)] =  mapbox_dict['mapbox']
    
color_axis['cmax'] = math.ceil(max_value * 1000) / 1000

len_of_max = len(str(color_axis['cmax']))
# tickvalues = np.around(np.linspace(0, np.round(color_axis['cmax'], len_of_max-1), 7), 3)
tickvalues = np.around(np.linspace(0, np.round(color_axis['cmax'], len_of_max), 7), 3)
ticktext = [str(val) for val in tickvalues]

color_axis['colorbar']['tickvals'] = tickvalues
color_axis['colorbar']['ticktext'] = ticktext

In [50]:
fig.update_layout(mapbox_dict)

fig.update_layout(
    title={"text": "Fuel Consumption", 'yanchor': 'middle', 'xanchor': 'auto'},
    hovermode='closest',
    coloraxis=color_axis
)

fig.show('browser')
pio.write_json(fig, os.path.join(definitions.ROOT, 'raw_plots', 'mapbox.plotly'))

In [53]:
x = np.array(binned_emissions_dict[time[0]][time[1]]['data']) == np.array(decel_emissions_dict[time[0]][time[1]]['data'])

c:\users\spraychamber\us69-research-paper-plotting\venv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning:

elementwise comparison failed; this will raise an error in the future.



In [54]:
x

False